# CS5242

## Load Data

In [5]:
from data import Data

train_data = Data(data_type='train', max_files=100)
test_data = Data(data_type='test',max_files=100)

print("Train data: x:"+ str(train_data.x.shape)+ ' y:'+str(train_data.y.shape))
print("Test data: x:"+ str(test_data.x.shape)+ ' y:'+str(test_data.y.shape))



Loading train data 100/100: 100%|██████████ [ time left: 00:00 ]
Loading test data 100/100: 100%|██████████ [ time left: 00:00 ]

Train data: x:(100, 1000, 102) y:(100, 1)
Test data: x:(100, 1000, 102) y:(100, 1)


## Model

In [7]:
import tensorflow as tf

model = tf.keras.Sequential([
    # tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:522: FutureWarning: Passing (type, 1) or 

In [8]:
history = model.fit(x=train_data.x, y=train_data.y, epochs=5)


Epoch 1/5
100/100 [==============================] - 4s 42ms/step - loss: 0.6758 - acc: 0.6300
Epoch 2/5
100/100 [==============================] - 3s 32ms/step - loss: 0.6510 - acc: 0.7600
Epoch 3/5
100/100 [==============================] - 3s 34ms/step - loss: 0.6302 - acc: 0.8100
Epoch 4/5
100/100 [==============================] - 4s 36ms/step - loss: 0.6110 - acc: 0.8200
Epoch 5/5
100/100 [==============================] - 4s 36ms/step - loss: 0.5940 - acc: 0.8300
